In [1]:




!pip install moviepy==2.0.0.dev2
!pip install imageio==2.25.1

!apt install imagemagick

!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

!pip install pysrt==1.1.2

import os
%pip install AssemblyAI
import assemblyai as aai

# Set your AssemblyAI API key
aai.settings.api_key = "fa08fc0604e44c31b52f69e79e280248"

# Initialize the transcriber
transcriber = aai.Transcriber()

# Specify the path to your video file
video_path = "newstrial.mp4"

# Extract the base name of the video file
video_base_name = os.path.splitext(os.path.basename(video_path))[0]

# Transcribe the audio from the video
transcript = transcriber.transcribe(video_path)

# Export the subtitles to an SRT file
subtitles = transcript.export_subtitles_srt()
srt_file_path = f"{video_base_name}.srt"

# Write the subtitles to the SRT file
with open(srt_file_path, "w") as f:
    f.write(subtitles)

print(f"Subtitles saved to: {srt_file_path}")


# Place files in this path or modify the paths to point to where the files are
srtfilename = srt_file_path
mp4filename = "newstrial.mp4"

import sys
import pysrt
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip


def time_to_seconds(time_obj):
    return time_obj.hours * 3600 + time_obj.minutes * 60 + time_obj.seconds + time_obj.milliseconds / 1000


def create_subtitle_clips(subtitles, videosize,fontsize=24, font='Arial', color='yellow', debug = False):
    subtitle_clips = []

    for subtitle in subtitles:
        start_time = time_to_seconds(subtitle.start)
        end_time = time_to_seconds(subtitle.end)
        duration = end_time - start_time

        video_width, video_height = videosize

        text_clip = TextClip(subtitle.text, fontsize=fontsize, font=font, color=color, bg_color = 'black',size=(video_width*3/4, None), method='caption').set_start(start_time).set_duration(duration)
        subtitle_x_position = 'center'
        subtitle_y_position = video_height* 4 / 5

        text_position = (subtitle_x_position, subtitle_y_position)
        subtitle_clips.append(text_clip.set_position(text_position))

    return subtitle_clips





# Load video and SRT file
video = VideoFileClip(mp4filename)
subtitles = pysrt.open(srtfilename)

begin,end= mp4filename.split(".mp4")
output_video_file = begin+'_subtitled'+".mp4"

print ("Output file name: ",output_video_file)

# Create subtitle clips
subtitle_clips = create_subtitle_clips(subtitles,video.size)

# Add subtitles to the video
final_video = CompositeVideoClip([video] + subtitle_clips)

# Write output video file
final_video.write_videofile(output_video_file)


from IPython.display import HTML
from base64 import b64encode
mp4 = open(output_video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)